# Extração de Frames
Este notebook realiza a extração de frames dos vídeos e os salva em diretórios específicos.

In [3]:
import os
import cv2
import pandas as pd

def extrair_frames(video_path, output_dir, frame_rate=15, frame_size=(224, 224)):
    """
    Extrai frames de um vídeo, redimensiona e os salva em uma pasta,
    mas apenas se o vídeo estiver listado no OSATS_clean.xlsx.

    Args:
        video_path (str): Caminho para o arquivo de vídeo.
        output_dir (str): Diretório onde os frames serão salvos.
        frame_rate (int): Taxa de quadros (frames) a serem extraídos por segundo.
        frame_size (tuple): Resolução desejada dos frames (largura, altura).
    """
    # Verificar se o vídeo está presente no OSATS_clean.xlsx
    allowed_df = pd.read_excel("OSATS_clean.xlsx")
    allowed_videos = set(allowed_df['VIDEO'].astype(str))
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    if video_id not in allowed_videos:
        print(f"Vídeo '{video_id}' não está presente no OSATS_clean.xlsx. Pulando.")
        return

    if os.path.exists(output_dir):
        print(f"Pasta '{output_dir}' já existe. Saltando este vídeo.")
        return

    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            resized_frame = cv2.resize(frame, frame_size)
            frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_path, resized_frame)
            frame_count += 1
        count += 1

    cap.release()
    print(f"Frames extraídos de {video_path} e salvos em {output_dir}")

In [4]:
# Exemplo de uso
VIDEO_PATH = "data/cropped_videos"
output_base_dir = "frames"

for video_file in os.listdir(VIDEO_PATH):
    if video_file.endswith(('.mp4')):
        video_path = os.path.join(VIDEO_PATH, video_file)
        output_dir = os.path.join(output_base_dir, os.path.splitext(video_file)[0])
        extrair_frames(video_path, output_dir, frame_rate=15, frame_size=(224, 224))

Frames extraídos de data/cropped_videos/U79I.mp4 e salvos em frames/U79I
Frames extraídos de data/cropped_videos/D44W.mp4 e salvos em frames/D44W
Frames extraídos de data/cropped_videos/T45Y.mp4 e salvos em frames/T45Y
Frames extraídos de data/cropped_videos/D12Y.mp4 e salvos em frames/D12Y
Frames extraídos de data/cropped_videos/E04A.mp4 e salvos em frames/E04A
Frames extraídos de data/cropped_videos/O47N.mp4 e salvos em frames/O47N
Frames extraídos de data/cropped_videos/G21U.mp4 e salvos em frames/G21U
Frames extraídos de data/cropped_videos/S77O.mp4 e salvos em frames/S77O
Frames extraídos de data/cropped_videos/N84Q.mp4 e salvos em frames/N84Q
Frames extraídos de data/cropped_videos/O96T.mp4 e salvos em frames/O96T
Frames extraídos de data/cropped_videos/Z93E.mp4 e salvos em frames/Z93E
Frames extraídos de data/cropped_videos/Q48N.mp4 e salvos em frames/Q48N
Frames extraídos de data/cropped_videos/X36T.mp4 e salvos em frames/X36T
Frames extraídos de data/cropped_videos/U24S.mp4 e 

In [3]:
import os
import cv2
import numpy as np
import pandas as pd

def extrair_frames_parcial(video_path, output_dir="frames_parcial", segment_duration=3,
                            target_frames=100, frame_size=(224,224)):
    """
    Extrai frames parciais de um vídeo dividindo-o em 4 partes e extraindo um segmento
    de 2-3 segundos em cada parte. São gerados cerca de 100 frames por vídeo
    (aproximadamente 25 por segmento), mas apenas se o vídeo estiver listado no OSATS_clean.xlsx.

    Args:
        video_path (str): Caminho para o arquivo de vídeo.
        output_dir (str): Diretório onde os frames serão salvos (padrão "frames_parcial").
        segment_duration (float): Duração do segmento a ser extraído de cada parte, em segundos (padrão: 3).
        target_frames (int): Número total desejado de frames (padrão: 100).
        frame_size (tuple): Tamanho (largura, altura) para redimensionar os frames.
    """
    # Verificar se o vídeo está presente no OSATS_clean.xlsx
    allowed_df = pd.read_excel("OSATS_clean.xlsx")
    allowed_videos = set(allowed_df['VIDEO'].astype(str))
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    if video_id not in allowed_videos:
        print(f"Vídeo '{video_id}' não está presente no OSATS_clean.xlsx. Pulando.")
        return

    if os.path.exists(output_dir):
        print(f"Pasta '{output_dir}' já existe. Saltando este vídeo.")
        return

    os.makedirs(output_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Não foi possível abrir o vídeo: {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps

    divisions = 4
    quarter_duration = video_duration / divisions
    frames_per_segment = target_frames // divisions  # aproximadamente 25 frames por segmento
    overall_frame_count = 0

    for i in range(divisions):
        quarter_start = i * quarter_duration
        
        # Centraliza o segmento dentro da parte, se possível
        if quarter_duration > segment_duration:
            segment_start = quarter_start + (quarter_duration - segment_duration) / 2
        else:
            segment_start = quarter_start

        start_frame = int(segment_start * fps)
        segment_total_frames = int(segment_duration * fps)

        # Gera índices uniformemente distribuídos para extrair frames do segmento
        frame_indices = np.linspace(start_frame, start_frame + segment_total_frames - 1,
                                    frames_per_segment, dtype=int)

        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if not ret:
                print(f"Frame {idx} não pôde ser lido.")
                continue

            resized_frame = cv2.resize(frame, frame_size)
            frame_filename = f"{video_id}_part{i}_frame{overall_frame_count:04d}.jpg"
            frame_path = os.path.join(output_dir, frame_filename)
            cv2.imwrite(frame_path, resized_frame)
            overall_frame_count += 1

    cap.release()
    print(f"Frames parciais extraídos de {video_path} e salvos em {output_dir}")

In [4]:
VIDEO_PATH = "data/cropped_videos"
output_base_dir = "frames_parcial"

for video_file in os.listdir(VIDEO_PATH):
    if video_file.endswith(('.mp4')):
        video_path = os.path.join(VIDEO_PATH, video_file)
        output_dir = os.path.join(output_base_dir, os.path.splitext(video_file)[0])
        extrair_frames_parcial(video_path, output_dir, target_frames=100, frame_size=(224, 224))


Frames parciais extraídos de data/cropped_videos/U79I.mp4 e salvos em frames_parcial/U79I
Frames parciais extraídos de data/cropped_videos/D44W.mp4 e salvos em frames_parcial/D44W
Frames parciais extraídos de data/cropped_videos/T45Y.mp4 e salvos em frames_parcial/T45Y
Frames parciais extraídos de data/cropped_videos/D12Y.mp4 e salvos em frames_parcial/D12Y
Frames parciais extraídos de data/cropped_videos/E04A.mp4 e salvos em frames_parcial/E04A
Frames parciais extraídos de data/cropped_videos/O47N.mp4 e salvos em frames_parcial/O47N
Frames parciais extraídos de data/cropped_videos/G21U.mp4 e salvos em frames_parcial/G21U
Frames parciais extraídos de data/cropped_videos/S77O.mp4 e salvos em frames_parcial/S77O
Frames parciais extraídos de data/cropped_videos/N84Q.mp4 e salvos em frames_parcial/N84Q
Frames parciais extraídos de data/cropped_videos/O96T.mp4 e salvos em frames_parcial/O96T
Frames parciais extraídos de data/cropped_videos/Z93E.mp4 e salvos em frames_parcial/Z93E
Frames par